# Full RAG Pipeline (Retrieval-Augmented Generation)

## 🎯 Bu Kodun Amacı
Tam bir RAG sistemi kurmak: Doküman → Embedding → VectorStore → Retrieval → Generation.

## Ne Yapacağız?
- PDF dokümanı yükleyip chunking yapacağız
- **HuggingFace Embeddings** ile metinleri vektörlere dönüştüreceğiz
- **ChromaDB** ile vector database oluşturacağız
- **Retriever** ile soruya en uygun chunk'ları bulacağız
- **RAG Chain** ile context'i LLM'e göndereceğiz
- Test soruları ile sistemin akademik makaleden cevap üretmesini göreceğiz

## Çalıştırma
Kod çalıştırıldığında tam bir RAG pipeline kurulur ve "Attention is All You Need" makalesi hakkında sorular sorulur.

## 📦 Gerekli Kütüphaneleri İçe Aktarma

**Ne yapıyoruz?**
- RAG sistemi için gerekli tüm LangChain bileşenlerini yüklüyoruz
- Groq LLM, PDF okuyucu, chunking aracı, embedding modeli ve vektör veritabanı

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

load_dotenv()

print("RAG PIPELINE KURULUMU\n")

RAG PIPELINE KURULUMU



## 📄 Adım 1/6: PDF Dokümanını Yükleme

**Ne yapıyoruz?**
- "Attention is All You Need" makalesini PyPDFLoader ile yüklüyoruz
- Her sayfa ayrı bir Document objesi olarak yüklenir
- Her Document: `page_content` (metin) + `metadata` (sayfa no, kaynak) içerir

**Sonuç:** 15 sayfalık PDF → 15 Document objesi

In [2]:
print("[1/6] Döküman yükleniyor...")
loader = PyPDFLoader("attention_is_all_you_need.pdf")
documents = loader.load()
print(f"      {len(documents)} sayfa yüklendi")

[1/6] Döküman yükleniyor...
      15 sayfa yüklendi


## ✂️ Adım 2/6: Metni Parçalara Ayırma (Chunking)

**Ne yapıyoruz?**
- Uzun metinleri **1000 karakterlik** parçalara bölüyoruz
- **100 karakter overlap** (örtüşme) kullanıyoruz

**Neden?**
- LLM'ler ve embedding modelleri sınırlı boyutla çalışır
- Overlap sayesinde cümleler kesilmez, context korunur

**chunk_size=1000:** Her parça maksimum 1000 karakter
**chunk_overlap=100:** Yan yana chunk'lar arasında 100 karakter ortak alan

**Örnek:**
- Chunk 1: [0-1000]
- Chunk 2: [900-1900] ← 100 karakterlik overlap

In [3]:
print("[2/6] Chunking işlemi...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)
print(f"      {len(chunks)} chunk oluşturuldu")

[2/6] Chunking işlemi...
      49 chunk oluşturuldu


## 🧮 Adım 3/6: Embedding Modelini Yükleme

**Ne yapıyoruz?**
- Metinleri **sayısal vektörlere** çeviren bir model yüklüyoruz
- Model: sentence-transformers/all-MiniLM-L6-v2 (384 boyutlu vektör)

**Embedding nedir?**
- Her metin → 384 uzunluğunda bir sayı dizisi
- Anlamca benzer metinler → benzer vektörler
- Örnek: 
  - "AI güçlü" → [0.2, 0.8, 0.1, ...]
  - "Yapay zeka kuvvetli" → [0.19, 0.82, 0.09, ...] ← çok benzer!

**Bu sayede:** Semantik (anlamsal) benzerlik hesaplayabiliriz

In [4]:
print("[3/6] Embedding modeli yükleniyor...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("      Embedding modeli hazır")

[3/6] Embedding modeli yükleniyor...
      Embedding modeli hazır


## 💾 Adım 4/6: Vektör Veritabanı Oluşturma (ChromaDB)

**Ne yapıyoruz?**
- Tüm chunk'ları embedding'lere çevirip ChromaDB'ye kaydediyoruz
- ChromaDB: Vektör benzerliği araması yapabilen özel bir veritabanı

**Süreç:**
1. Her chunk → Embedding modeline gönderiliyor
2. Embedding (vektör) + chunk metni + metadata → ChromaDB'ye kaydediliyor
3. Toplam 49 chunk veritabanında

**Veritabanında ne var?**
```
[
  {id: 1, text: "Attention Is...", vector: [0.2, 0.8, ...], metadata: {page: 0}},
  {id: 2, text: "Transformer...", vector: [0.3, 0.7, ...], metadata: {page: 1}},
  ...
]
```

**Koleksiyon adı:** "attention_paper" (birden fazla koleksiyon olabilir)

In [5]:
print("[4/6] ChromaDB oluşturuluyor...")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="attention_paper"
)
print(f"      {vectorstore._collection.count()} chunk veritabanında")

[4/6] ChromaDB oluşturuluyor...
      49 chunk veritabanında


## 🔍 Adım 5/6: Retriever (Arama Motoru) Yapılandırma

**Ne yapıyoruz?**
- Veritabanından en benzer chunk'ları bulacak bir arama motoru oluşturuyoruz

**search_type="similarity":** Benzerlik araması yap
**search_kwargs={"k": 3}:** En benzer **3 chunk** getir

**Nasıl çalışır?**
1. Soru gelir: "Transformer nedir?"
2. Soru embedding'e çevrilir: [0.25, 0.75, ...]
3. Veritabanındaki tüm vektörlerle karşılaştırılır (cosine similarity)
4. En yüksek skor alan 3 chunk döner

**Neden k=3?**
- Çok az (k=1): Yeterli bilgi olmayabilir
- Çok fazla (k=10): Gereksiz bilgi ve yavaş performans

In [6]:
print("[5/6] Retriever yapılandırılıyor...")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)
print("      Retriever hazır")

[5/6] Retriever yapılandırılıyor...
      Retriever hazır


## 🔗 Adım 6/6: RAG Chain (Pipeline) Oluşturma

**Ne yapıyoruz?**
- Tüm bileşenleri birleştirip tam bir RAG zinciri oluşturuyoruz

**Bileşenler:**
1. **LLM:** ChatGroq - Cevap üreten model
2. **Prompt Template:** Modele nasıl davranacağını söyleyen şablon
3. **format_docs:** 3 chunk'ı birleştiren yardımcı fonksiyon
4. **RAG Chain:** Tüm akışı yöneten zincir

**RAG Chain Akışı (LCEL ile):**
```
Soru: "Transformer nedir?"
    ↓
{"context": retriever | format_docs, "question": "Transformer nedir?"}
    ↓ (retriever çalışır)
Veritabanından 3 benzer chunk bulunur
    ↓ (format_docs çalışır)
3 chunk birleştirilir: "Chunk1\n\nChunk2\n\nChunk3"
    ↓ (prompt template doldurulur)
"Sen bir asistansın... Bağlam: [3 chunk] Soru: Transformer nedir?"
    ↓ (LLM çalışır)
Model cevap üretir
    ↓ (StrOutputParser çalışır)
String olarak döner: "Transformer modeli..."
```

**LCEL Operatörleri:**
- `|` : "sonra" anlamına gelir (pipe)
- `RunnablePassthrough()`: Veriyi olduğu gibi ilet

In [7]:
print("[6/6] RAG chain oluşturuluyor...")
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.3,
    groq_api_key=os.getenv("GROQ_API_KEY")
)

template = """Sen bir akademik araştırma asistanısın. Verilen makale bölümüne göre soruyu cevapla.

Bağlam:
{context}

Soru: {question}

Kurallar:
- Sadece verilen makale içeriğine dayanarak cevap ver
- Cevap bulunamazsa "Bu bilgi makalede bulunmuyor" de
- Türkçe ve açık bir dille yaz

Cevap:"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("      RAG pipeline hazır\n")

[6/6] RAG chain oluşturuluyor...
      RAG pipeline hazır



In [8]:
test_questions = [
    "Transformer modeli nedir?",
    "Self-attention mekanizması nasıl çalışır?",
    "Bu makalede hangi veri setleri kullanılmış?"
]

print("TEST SORULARI\n")

for i, question in enumerate(test_questions, 1):
    print(f"SORU {i}: {question}")
    print("-" * 60)
    
    answer = rag_chain.invoke(question)
    
    print("CEVAP:")
    print(answer)
    print("\n")

TEST SORULARI

SORU 1: Transformer modeli nedir?
------------------------------------------------------------
CEVAP:
Transformer modeli, makalede belirtilen bir dil çevirme modelidir. İngilizce'den Almanca ve Fransızca'ya çeviri yapabilen bu model, önceki state-of-the-art modellere göre daha iyi BLEU puanlarına ulaşıyor ve daha düşük bir eğitim maliyeti gerektiriyor. Modelin farklı varyasyonları ve hiperparametreleri değiştirilerek performansı değerlendiriliyor. Ancak, Transformer modelinin ayrıntılı bir tanımı veya teknik açıklaması makalede bulunmuyor. Sadece modelin performansı ve varyasyonları hakkında bilgi veriliyor.


SORU 2: Self-attention mekanizması nasıl çalışır?
------------------------------------------------------------
CEVAP:
Self-attention mekanizması, bir dizi içindeki farklı pozisyonları ilişkilendirerek dizi hakkında bir temsil oluşturur. Bu, bir dizi içindeki uzun mesafeli bağımlılıkları öğrenmeye yardımcı olur. Makalede verilen örnekte, self-attention mekanizması, 